# **Supervised Learning code for Disaster Tweet Classification**

✔✔✔

Installing the libraries


**Multiclass Text Classification Using BERT and Keras****
we will use ktrain (a lightweight wrapper around Keras) to build a model using the dataset employed in the scikit-learn tutorial: Working with Text Data.The objective is to accurately classify each text into one of these 2 classes. This will provide us an opportunity to see BERT in action on a relatively smaller training set.

In [ ]:
!pip3 install ktrain==0.2.2

### Importing Libraries


In [ ]:
from sklearn.model_selection import train_test_split

### Reading training file

In [ ]:
import pandas as pd
train_df = pd.read_csv("../input/nlp-getting-started/train.csv")

In [ ]:
train_df.head()

### Splitting training data in to training and validation data

In [ ]:
random_seed = 12342
x_train, x_val, y_train, y_val = train_test_split(train_df['text'], train_df['target'], shuffle=True, test_size = 0.2, random_state=random_seed, stratify=train_df['target'])

**Convert data to features for BERT**
# **Load and Preprocess the Data**
Preprocess the data using the texts_from_array function (since the data resides in an array). If your documents are stored in folders or a CSV file you can use the texts_from_folder or texts_from_csv functions, respectively.

In [ ]:
import ktrain
from ktrain import text

In [ ]:
(x_train_bert,  y_train_bert), (x_val_bert, y_val_bert), preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                                                                         x_test = x_val, y_test=y_val,
                                                                                          class_names= ["0", "1"],
                                                                                          preprocess_mode='bert',
                                                                                          
                                                                                          maxlen=65, 
                                                                                          max_features=35000)

# STEP 2: Load the BERT Model and Instantiate a Learner object
**Create Model Instance and train it for 6 epochs
**
# STEP 3: Train the Model
We train using one of the three learning rates recommended in the BERT paper: 5e-5, 3e-5, or 2e-5. Alternatively, the ktrain Learning Rate Finder can be used to find a good learning rate by invoking learner.lr_find() and learner.lr_plot(), prior to training. The learner.fit_onecycle method employs a 1cycle learning rate policy.



In [ ]:
model = text.text_classifier('bert', train_data=(x_train_bert, y_train_bert), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train_bert, y_train_bert), val_data=(x_val_bert, y_val_bert), batch_size=16)

In [ ]:
learner.lr_find()             # briefly simulate training to find good learning rate
   

In [ ]:
learner.lr_plot()

In [ ]:
learner.autofit(1e-5)

In [ ]:
learner.validate(val_data=(x_val_bert, y_val_bert), class_names=['No Disaster', 'Disaster'])

### Getting predictor variable

In [ ]:
# getting predictor variable
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
learner.print_layers()

### Getting predictions on test data

In [ ]:
test_df = pd.read_csv("../input/nlp-getting-started/test.csv")
test_df["target"] = predictor.predict(test_df["text"].tolist())

test_df = test_df[["id", "target"]]


In [ ]:
test_df.head()

In [ ]:
test_df.to_csv("submisssions.csv", index=False)